In [2]:
import numpy as np
import pandas as pd

In [3]:
data=pd.read_csv('titanic3.csv')
y=data['survived']
X=data.copy()
del X['survived']

Missing Values

In [4]:
data.isnull().mean().sort_values(ascending=True)

pclass       0.000000
survived     0.000000
name         0.000000
sex          0.000000
sibsp        0.000000
parch        0.000000
ticket       0.000000
fare         0.000764
embarked     0.001528
age          0.200917
home.dest    0.430863
boat         0.628724
cabin        0.774637
body         0.907563
dtype: float64

In [5]:
#body
X['body_null']=np.where(X.body.isnull(),1,0)
X.body.fillna(0,inplace=True)

In [6]:
#cabin
temp_cabin=X.cabin.str.split(expand=True)[0]
X['cabin_char']=temp_cabin.str[0]
#X['cabin_num']=temp_cabin.str[1:]
del X['cabin']

X['cabin_char']=np.where(X.cabin_char.isnull(),'M',X.cabin_char)

In [7]:
#boat
temp_boat=X.boat.str.rsplit(expand=True).rename(columns={0:'boat_1',1:'boat_2',2:'boat_3'})
X['boat_char']=np.where(temp_boat.boat_1.str.isdigit(),np.nan,temp_boat.boat_1)
#X['boat_num']=np.where(temp_boat.boat_1.str.isdigit(),temp_boat.boat_1,np.nan)

X['boat_char']=np.where(X.boat_char.isnull(),'M',X.boat_char)
#X['boat_num'].fillna(0,inplace=True)
del X['boat']

In [8]:
#home.dest
del X['home.dest']

#age
X.age.fillna(X.age.mean(),inplace=True)

#Embarked
X['embarked']=np.where(X.embarked.isnull(),'M',X.embarked)

#fare
X.fare.fillna(X.age.median(),inplace=True)

In [9]:
#LabelEncoding
#name
X['name']=X.name.str.split(',',expand=True)[1].str.split('.',expand=True)[0]
X['name']=X['name'].map({' Mrs': 0,  ' Mr': 1, ' Miss': 2, ' Master': 3, ' Col': 1, ' Mme': 0, ' Dr': 4, ' Major': 1, ' Capt': 1, ' Lady': 0, ' Sir': 1, ' Mlle': 0, ' Dona': 0, ' Jonkheer': 1, ' the Countess': 0, ' Don': 1, ' Rev': 1, ' Ms': 0})

#sex
X['sex']=X['sex'].map({'male':0,'female':1})

del X['ticket']

#Embarked
X['embarked']=X.embarked.map({'M':0,'S':1,'C':2,'Q':3})


#Cabin_char
X['cabin_char']=X.cabin_char.map({k:i for i,k in enumerate(X.cabin_char.unique())})

#Boat_char
X['boat_char']=X.boat_char.map({k:i for i,k in enumerate(X.boat_char.unique())})


In [10]:
#One hot encoding

#name
temp_name=pd.get_dummies(X.name,drop_first=True)
X=pd.concat([X,temp_name],axis=1)
del X['name']

#embarked
temp_embarked=pd.get_dummies(X.embarked,drop_first=True).rename(columns={1:5,2:6,3:7})
X=pd.concat([X,temp_embarked],axis=1)
del X['embarked']


In [11]:
## train test split

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y,random_state=50)

# Feature Selection

## Filter Methods

In [12]:
#Basic Methods
X_train_filter=X_train.copy()
y_train_filter=y_train.copy()
X_test_filter=X_test.copy()
y_test_filter=y_test.copy()

from sklearn.feature_selection import VarianceThreshold
sel=VarianceThreshold(threshold=0.01)
sel.fit(X_train_filter)
sel.transform(X_train)
X_train_filter.columns[sel.get_support()]
del X_train_filter[4]
del X_test_filter[4]

from xgboost import XGBClassifier
model_filter=XGBClassifier()
model_filter.fit(X_train_filter,y_train_filter)


y_pred_filter=model_filter.predict(X_test_filter)

from sklearn.metrics import confusion_matrix
metric_filter=confusion_matrix(y_test_filter,y_pred_filter)

accuracy_filter_basic=(metric_filter[0][0]+metric_filter[1][1])/sum(sum(metric_filter))*100
print('Accuracy using filter : ',accuracy_filter_basic)

Accuracy using filter :  84.7328244274809


In [13]:
#Correlation Methods
X_train_filter=X_train.copy()
y_train_filter=y_train.copy()
X_test_filter=X_test.copy()
y_test_filter=y_test.copy()

corrmat={}
for i in X_train_filter.columns.values:
    corrmat[i]=X_train_filter[i].corr(y_train_filter)
del X_train_filter[7]
del X_train_filter[4]
del X_train_filter[3]
del X_train_filter['parch']
del X_train_filter['sibsp']
del X_train_filter['age']
del X_test_filter[7]
del X_test_filter[4]
del X_test_filter[3]
del X_test_filter['parch']
del X_test_filter['sibsp']
del X_test_filter['age']

from xgboost import XGBClassifier
model_filter=XGBClassifier()
model_filter.fit(X_train_filter,y_train_filter)


y_pred_filter=model_filter.predict(X_test_filter)

from sklearn.metrics import confusion_matrix
metric_filter=confusion_matrix(y_test_filter,y_pred_filter)

accuracy_filter_basic=(metric_filter[0][0]+metric_filter[1][1])/sum(sum(metric_filter))*100
print('Accuracy using filter : ',accuracy_filter_basic)

Accuracy using filter :  85.87786259541986


#### Statistical Filter Methods

In [14]:
#Information gain / Mutual information
X_train_filter=X_train.copy()
y_train_filter=y_train.copy()
X_test_filter=X_test.copy()
y_test_filter=y_test.copy()

from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.feature_selection import SelectKBest,SelectPercentile
#mi=mutual_info_classif(X_train_filter,y_train_filter)
select=SelectKBest(mutual_info_classif,k=12).fit(X_train_filter,y_train_filter)
X_train_filter=X_train_filter[X_train_filter.columns[select.get_support()].values]
X_test_filter=X_test_filter[X_test_filter.columns[select.get_support()].values]
from xgboost import XGBClassifier
model_filter=XGBClassifier()
model_filter.fit(X_train_filter,y_train_filter)


y_pred_filter=model_filter.predict(X_test_filter)

from sklearn.metrics import confusion_matrix
metric_filter=confusion_matrix(y_test_filter,y_pred_filter)

accuracy_filter_basic=(metric_filter[0][0]+metric_filter[1][1])/sum(sum(metric_filter))*100
print('Accuracy using filter : ',accuracy_filter_basic)

Accuracy using filter :  84.7328244274809


In [15]:
#Fisher score / chi square
#Information gain / Mutual information
X_train_filter=X_train.copy()
y_train_filter=y_train.copy()
X_test_filter=X_test.copy()
y_test_filter=y_test.copy()

from sklearn.feature_selection import SelectKBest, chi2
select=SelectKBest(chi2,k=10).fit(X_train_filter,y_train_filter)
X_train_filter=X_train_filter[X_train_filter.columns[select.get_support()].values]
X_test_filter=X_test_filter[X_test_filter.columns[select.get_support()].values]
from xgboost import XGBClassifier
model_filter=XGBClassifier()
model_filter.fit(X_train_filter,y_train_filter)


y_pred_filter=model_filter.predict(X_test_filter)

from sklearn.metrics import confusion_matrix
metric_filter=confusion_matrix(y_test_filter,y_pred_filter)

accuracy_filter_basic=(metric_filter[0][0]+metric_filter[1][1])/sum(sum(metric_filter))*100
print('Accuracy using filter : ',accuracy_filter_basic)

Accuracy using filter :  85.49618320610686


In [16]:
# Univariate / ANOVA
X_train_filter=X_train.copy()
y_train_filter=y_train.copy()
X_test_filter=X_test.copy()
y_test_filter=y_test.copy()

from sklearn.feature_selection import SelectKBest, f_classif
select=SelectKBest(f_classif,k=11).fit(X_train_filter,y_train_filter)
X_train_filter=X_train_filter[X_train_filter.columns[select.get_support()].values]
X_test_filter=X_test_filter[X_test_filter.columns[select.get_support()].values]
from xgboost import XGBClassifier
model_filter=XGBClassifier()
model_filter.fit(X_train_filter,y_train_filter)


y_pred_filter=model_filter.predict(X_test_filter)

from sklearn.metrics import confusion_matrix
metric_filter=confusion_matrix(y_test_filter,y_pred_filter)

accuracy_filter_basic=(metric_filter[0][0]+metric_filter[1][1])/sum(sum(metric_filter))*100
print('Accuracy using filter : ',accuracy_filter_basic)

Accuracy using filter :  85.87786259541986


## Wrapper methods

In [17]:
#Forward feature selection
X_train_wrapper=X_train.copy()
y_train_wrapper=y_train.copy()
X_test_wrapper=X_test.copy()
y_test_wrapper=y_test.copy()
from xgboost import XGBClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
sfs1=SFS(XGBClassifier(n_jobs=4),k_features=9,forward=True,floating=False,verbose=2,scoring='roc_auc',cv=3)
sfs1.fit(np.array(X_train_wrapper),y_train_wrapper)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    1.6s finished

[2019-03-21 00:39:01] Features: 1/9 -- score: 0.7701856308627381[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    1.6s finished

[2019-03-21 00:39:02] Features: 2/9 -- score: 0.8362594283799153[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    1.7s finished

[2019-03-21 00:39:04] Features: 3/9 -- score: 0.8639585160385762[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 

SequentialFeatureSelector(clone_estimator=True, cv=3,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=4, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
             floating=False, forward=True, k_features=9, n_jobs=1,
             pre_dispatch='2*n_jobs', scoring='roc_auc', verbose=2)

In [18]:
#Backward feature selection
X_train_wrapper=X_train.copy()
y_train_wrapper=y_train.copy()
X_test_wrapper=X_test.copy()
y_test_wrapper=y_test.copy()
from xgboost import XGBClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
sfs1=SFS(XGBClassifier(n_jobs=4),k_features=11,forward=False,floating=False,verbose=2,scoring='roc_auc',cv=3)
sfs1.fit(np.array(X_train_wrapper),y_train_wrapper)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    2.5s finished

[2019-03-21 00:39:15] Features: 16/11 -- score: 0.8954140182562026[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    2.8s finished

[2019-03-21 00:39:18] Features: 15/11 -- score: 0.898289816500705[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    4.2s finished

[2019-03-21 00:39:22] Features: 14/11 -- score: 0.8995550077177207[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done

SequentialFeatureSelector(clone_estimator=True, cv=3,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=4, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
             floating=False, forward=False, k_features=11, n_jobs=1,
             pre_dispatch='2*n_jobs', scoring='roc_auc', verbose=2)

In [ ]:
#Exhaustive feature selection
X_train_wrapper=X_train.copy()
y_train_wrapper=y_train.copy()
X_test_wrapper=X_test.copy()
y_test_wrapper=y_test.copy()
from xgboost import XGBClassifier
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
sfs1=EFS(XGBClassifier(n_jobs=4),min_features=4,max_features=5,scoring='roc_auc',print_progress=True,cv=2)
sfs1.fit(np.array(X_train_wrapper),y_train_wrapper)

## Embedded methods

In [35]:
X_train_embedded=X_train.copy()
y_train_embedded=y_train.copy()
X_test_embedded=X_test.copy()
y_test_embedded=y_test.copy()
del X_train_embedded['body_null']
del X_train_embedded[2]
del X_train_embedded[3]
del X_train_embedded[7]
del X_test_embedded['body_null']
del X_test_embedded[2]
del X_test_embedded[3]
del X_test_embedded[7]

from xgboost import XGBClassifier
xgb=XGBClassifier()
xgb.fit(X_train_embedded,y_train_embedded)
print(xgb.feature_importances_*100)

y_pred_embedded=xgb.predict(X_test_embedded)

from sklearn.metrics import confusion_matrix
metric_embedded=confusion_matrix(y_test_embedded,y_pred_embedded)

accuracy_embedded=(metric_embedded[0][0]+metric_embedded[1][1])/sum(sum(metric_embedded))*100
print('Accuracy using embedded : ',accuracy_embedded)

[11.880886   2.4750252  1.1847578  2.2915356  1.2289958  2.1646218
  4.18118    4.870522   8.314034  52.071716   5.089633   1.8037684
  2.4433258]
Accuracy using embedded :  84.7328244274809


,pclass,sex,age,sibsp,parch,fare,body,cabin_char,boat_char,1,4,5,6
1004,3,1,29.881135,0,0,7.7875,0.0,5,0,0,0,0,0
315,1,1,21.000000,0,0,26.5500,0.0,5,0,0,0,1,0
655,3,0,32.000000,1,0,15.8500,0.0,5,1,1,0,1,0
1261,3,1,63.000000,0,0,9.5875,0.0,5,0,0,0,1,0
884,3,0,48.000000,0,0,7.8542,0.0,5,0,1,0,1,0
1132,3,0,29.881135,0,0,7.8958,0.0,5,0,1,0,1,0
1005,3,0,29.881135,0,0,15.5000,0.0,5,0,1,0,0,0
764,3,0,26.000000,1,2,20.5750,0.0,5,0,1,0,1,0
1259,3,0,36.000000,0,0,7.8958,0.0,5,0,1,0,1,0
848,3,0,41.000000,2,0,14.1083,0.0,5,0,1,0,1,0
